# Notebook para pruebas y aprender data visualization

Este notebook lo he creado para hacer pruebas de visualización de datos. Como hasta ahora solamente había hecho cursos y ejercicios guiados he creado este notebook para ir respondiendo preguntas que me hago sobre un dataset por mi cuenta sin supervisión.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        
df = pd.read_csv(filepath, index_col='Rank')
df.shape

In [ ]:
df.head()

## Plataformas disponibles en el dataset

In [ ]:
platforms = ""
for platform in df['Platform'].unique():
    platforms += platform + ", "
 
print(platforms[:-2]) # -2 in order to remove the last ',' character

## Cuantos juegos vendió Wii por año

In [ ]:
wii_games = df[df['Platform'] == 'Wii']
sales_per_year = wii_games.groupby('Year').Global_Sales.sum().to_frame()

sns.lineplot(x=sales_per_year.index, y='Global_Sales', data=sales_per_year)
plt.title('Total Wii sales')

## Comparativa de ventas de juegos por año

In [ ]:
sales_per_year_and_platform = df.groupby(['Year', 'Platform']).Global_Sales.sum().to_frame().reset_index()

plt.figure(figsize=(20,10))
plt.title('Sales per year and platform')
plt.xlabel('Year')
plt.ylabel('Sales')

sns.lineplot(data=sales_per_year_and_platform, x='Year', y='Global_Sales', hue='Platform')

La idea de esta grafica era buena. Ver una comparativa de ventas de todas las plataformas a lo largo del tiempo pero una vez calculado no se puede diferenciar cada plataforma. Los colores de la leyenda se repiten porque son muchas plataformas.

A pesar de esto, podemos observar que hubo tres puntos donde despuntaron las ventas, aproximadamente: 1997, entre 2002 y 2005 y 2010

### Mapa de calor

Los mapas de calor necesitan un conjunto de datos el cual todo su contenido sean números para poder comparar sus valores y asignar colores en función de esto. Dónde único se puede tener valores no numéricos es en el índice y la cabecera.

In [ ]:
heatmap_data = sales_per_year_and_platform.pivot(columns='Year', index='Platform', values='Global_Sales').fillna(0)
heatmap_data.head()

In [ ]:
plt.figure(figsize=(60,20))
plt.title('Sales per year and platform')
plt.xlabel('Year')
plt.ylabel('Sales')

sns.heatmap(heatmap_data, annot=True)

Como podemos ver en el gráfico anterior el groso de ventas de videojuegos ha sido por parte de las plataformas Wii, DS, X360, PS3, PS2, PS, N64, 3DS. Además de esto, vemos como estas ventas se centran desde el año 1995 hasta el 2017.

Por esta razón vamos a hacer un nuevo mapa de calor solamente con estos datos.

In [ ]:
reduced_sales_per_year_and_platform = sales_per_year_and_platform.query('Year>1994 and Year<2018 and (Platform == "Wii" or Platform == "DS" or Platform == "X360" or Platform == "PS3" or Platform == "PS2" or Platform == "PS" or Platform == "N64" or Platform == "3DS")')
heatmap_data = reduced_sales_per_year_and_platform.pivot(columns='Year', index='Platform', values='Global_Sales').fillna(0)
heatmap_data.head()

In [ ]:
heatmap_data['total_sales_to_sort'] = heatmap_data.T.sum()
heatmap_data = heatmap_data.sort_values('total_sales_to_sort', ascending=False)
del heatmap_data['total_sales_to_sort']
heatmap_data.head()

In [ ]:
plt.figure(figsize=(30,10))
plt.title('Sales per year and platform')
plt.xlabel('Year')
plt.ylabel('Sales')

# platforms_to_show = ['Wii', 'DS', 'X360', 'PS3', 'PS2', 'PS', 'N64', '3DS']
# years_to_show = range(1995, 2017)

sns.heatmap(heatmap_data, annot=True)

La query que hacemos para obtener estos datos no es nada "elegante" escribimos muchos or para las distintas plataformas. ¿Se puede mejorar de alguna forma?

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Sales per year and platform')
plt.xlabel('Year')
plt.ylabel('Sales')

sns.lineplot(data=reduced_sales_per_year_and_platform, x='Year', y='Global_Sales', hue='Platform')

Grafico de puntos por genero y ventas totales, agrupados por color dependiendo de la plataforma

Queremos saber si hay una relacion entre el genero y la plataforma. Para esto tenemos que agrupar las ventas globales por plataforma y genero. De tal manera, que solamente tengamos una entrada de datos con una plataforma y un genero, no debe repetirse esta combinacion.

In [ ]:
relevant_data = df.query('Year>1994 and Year<2018 and (Platform == "Wii" or Platform == "DS" or Platform == "X360" or Platform == "PS3" or Platform == "PS2" or Platform == "PS" or Platform == "N64" or Platform == "3DS")')
relevant_data.head()

In [ ]:
genre_platform_group = relevant_data.groupby(['Genre', 'Platform']).Global_Sales.sum().reset_index()
genre_platform_group.shape

Con la operación `groupby` juntamos por la unión de estos dos campos `Genre` `Platform`, y sumamos el resto de valores de la fila. En este caso, las ventas globales. Para evitar que cree un índice nuevo con estos dos campos por los que agrupamos usamos la función `reset_index`. De esta forma los campos por los que hacemos la unión quedan accesibles mediante su columna correspondiente.

In [ ]:
genre_platform_group.head()

## ¿El genero de acción vende igual en todas las plataformas?

In [ ]:
action_games = genre_platform_group[genre_platform_group['Genre'] == 'Action'].sort_values("Global_Sales", ascending=False)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Global sales of action games per paltform")
plt.ylabel("Global total sales")
plt.xlabel("Platform")

sns.barplot(data=action_games, x='Platform', y='Global_Sales')

## ¿En Wii vende todos los generos por igual?

In [ ]:
wii_games = genre_platform_group[genre_platform_group['Platform'] == 'Wii'].sort_values("Global_Sales", ascending=False)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Global sales of action games per paltform")
plt.ylabel("Global total sales")
plt.xlabel("Genre")

sns.barplot(data=wii_games, x='Genre', y='Global_Sales')

En este caso el pico de ventas en juegos de deportes en la plataforma Wii se debe a que el juego "Wii sports" se "vendia" junto con la consola.

## ¿Que genero destaca en cada plataforma?

In [ ]:
genre_platform_group.head()

In [ ]:
# funcion para obtener el genero que mas ventas tiene por plataforma

result = genre_platform_group.groupby('Platform').Global_Sales.max()

result.head()

## la misma grafica que para la wii por generos pero con todas las plataformas